In [1]:
# -----------------------------------------------------------------------------
# :author:    Juanjuan Huang & Shelly D. Keylly
# :email:     juanjuan.huang@anl.gov
# :copyright: Copyright © 2023, UChicago Argonne, LLC
# -----------------------------------------------------------------------------

## Imports

In [2]:
# %matplotlib widget
from neighbor_core import ClusterNeighbor
from ipyfilechooser import FileChooser
import numpy as np
from neighbor_core import fit_polynomial
import plotly.graph_objects as go


# Example 1, basic functionality go through

## Initialize a cluster and view it

In [3]:
fc = FileChooser()
display(fc)

FileChooser(path='/Users/juanjuanhuang/Desktop/neighbor', filename='', title='', show_hidden=False, select_des…

## view it

In [4]:
cluster = ClusterNeighbor()
if fc.value is not None:
    cluster.load_xyz(path=fc.value)
else:
    cluster.load_xyz(path="structure_examples/198087-ICSD molecule.xyz")
cluster.view_xyz(label=True, show_symbol=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Plot histogram

In [5]:
cluster.get_pairs()
cluster.plot_hist(plot_engine="plotly")

## Get CN reports and save to excel

In [6]:
cluster.get_CN_summary_all(tolerance=0.1, bond_range=5, printit=False)
cluster.print_CN_summary()
cluster.write_to_excel("20240806_test_original.xlsx")

Bond Type: Pb-O
-- Distance: 2.164006661818935
-- Average CN: 5.2
-- Info: 
4 atoms have CN of 3; 
11 atoms have CN of 6; 
-- Tolerance: 0.1 

-- Distance: 3.759464103600722
-- Average CN: 2.4
-- Info: 
12 atoms have CN of 2; 
3 atoms have CN of 4; 
-- Tolerance: 0.1 

-- Distance: 4.0161487745681885
-- Average CN: 2.4
-- Info: 
10 atoms have CN of 2; 
4 atoms have CN of 3; 
1 atoms have CN of 4; 
-- Tolerance: 0.1 

-- Distance: 4.450974934970656
-- Average CN: 4.266666666666667
-- Info: 
14 atoms have CN of 4; 
1 atoms have CN of 8; 
-- Tolerance: 0.1 

-- Distance: 4.861048023898767
-- Average CN: 0.9333333333333333
-- Info: 
4 atoms have CN of 0; 
8 atoms have CN of 1; 
3 atoms have CN of 2; 
-- Tolerance: 0.1 

Bond Type: Pb-Pb
-- Distance: 3.389999918388575
-- Average CN: 0.8
-- Info: 
4 atoms have CN of 0; 
10 atoms have CN of 1; 
1 atoms have CN of 2; 
-- Tolerance: 0.1 

-- Distance: 3.895359489312166
-- Average CN: 4.266666666666667
-- Info: 
14 atoms have CN of 4; 
1 atoms h

## remove undercoordinated atoms

In [7]:
cluster.remove_under_coordinated_atoms(center_atom="Pb", CN_atom="O", CN_threshold=4, bond_range=3, self_apply=True)
cluster.view_xyz(label=False, show_symbol=False)

Remove under-coordinated atoms: Atoms(symbols='Pb4', pbc=False)[[22, 37, 57, 58]]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
cluster.get_CN_summary_all(tolerance=0.1, bond_range=5, printit=False)
cluster.print_CN_summary()
cluster.write_to_excel("20240807_test_reduced.xlsx")

Bond Type: Pb-O
-- Distance: 2.164006684677769
-- Average CN: 6.0
-- Info: 
11 atoms have CN of 6; 
-- Tolerance: 0.1 

-- Distance: 3.7594640623910167
-- Average CN: 2.5454545454545454
-- Info: 
8 atoms have CN of 2; 
3 atoms have CN of 4; 
-- Tolerance: 0.1 

-- Distance: 4.016148722271057
-- Average CN: 2.5454545454545454
-- Info: 
6 atoms have CN of 2; 
4 atoms have CN of 3; 
1 atoms have CN of 4; 
-- Tolerance: 0.1 

-- Distance: 4.450974886404176
-- Average CN: 4.363636363636363
-- Info: 
10 atoms have CN of 4; 
1 atoms have CN of 8; 
-- Tolerance: 0.1 

-- Distance: 4.861047941704461
-- Average CN: 0.9090909090909091
-- Info: 
4 atoms have CN of 0; 
4 atoms have CN of 1; 
3 atoms have CN of 2; 
-- Tolerance: 0.1 

Bond Type: Pb-Pb
-- Distance: 3.389999918388575
-- Average CN: 1.0909090909090908
-- Info: 
10 atoms have CN of 1; 
1 atoms have CN of 2; 
-- Tolerance: 0.1 

-- Distance: 3.8953594869263837
-- Average CN: 2.909090909090909
-- Info: 
8 atoms have CN of 2; 
2 atoms have

# Example 2: Polynomial fits of different shells

## select a cif file

In [9]:
fc = FileChooser()
display(fc)

FileChooser(path='/Users/juanjuanhuang/Desktop/neighbor', filename='', title='', show_hidden=False, select_des…

In [10]:
cif_cluster = ClusterNeighbor()
if fc.value is not None:
    cif_cluster.load_xyz(path=fc.value)
else:
   cif_cluster.load_xyz(path="test_Pt/Ptfoil_243678.cif")
cif_cluster.view_xyz(label=True, show_symbol=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
cif_cluster.get_CN_all(tolerance=0.1, bond_range=10, printit=True)

Pt-Pt
1 length: 2.774  CN: 1968.000


## For multiple shells with undercoordinated atoms removed

In [12]:
center_atom = "Pt"
CN_atom = "Pt"
CN_bond_range = 6
tolerance = 0.1
undercoordinated_threshold = 4

# Initialize dictionaries to store results
CNs_all = {}
bondlengths_all = {}
num_atoms_all = {}
sizes_all = {}

# sizes = np.hstack([np.arange(5, 20, 2), np.arange(20, 80, 10)])
sizes = np.hstack([np.arange(5, 20, 5), np.arange(20, 40, 10)])

for i in sizes:
    expanded_cluster = cif_cluster.expand_to_sphere(target_diameter=i, self_apply=False)
    new_cluster = ClusterNeighbor()
    new_cluster.load_xyz(from_file=False, atom_object=expanded_cluster)
    num_atoms = len(new_cluster.atoms)
    new_cluster.atoms.set_pbc(False)

    if 4 < num_atoms < 100:
        new_cluster.remove_under_coordinated_atoms(center_atom=center_atom, 
                                                   CN_atom=CN_atom, 
                                                   CN_threshold=undercoordinated_threshold, 
                                                   bond_range=5, 
                                                   self_apply=True)
    
    new_cluster.get_CN_summary_all(tolerance=tolerance, 
                                   bond_range=CN_bond_range, 
                                   printit=False)
    
    # Get bond lengths for each shell
    bond_lengths = list(new_cluster.CN_summary[f'{center_atom}-{CN_atom}'].keys())
    new_cluster.write_to_excel(f"size_{new_cluster.get_cluster_size():.2f}A")
    # new_cluster.print_CN_summary()
    for shell_index, bond_length in enumerate(bond_lengths):
        shell_key = f'shell_{shell_index + 1}'
        
        # Ensure the key exists in each dictionary
        if shell_key not in CNs_all:
            CNs_all[shell_key] = []
            bondlengths_all[shell_key] = []
            num_atoms_all[shell_key] = []
            sizes_all[shell_key] = []
        
        # Append data to the relevant shell
        bondlengths_all[shell_key].append(bond_length)
        CNs_all[shell_key].append(new_cluster.CN_summary[f'{center_atom}-{CN_atom}'][bond_length]['average_CN'])
        sizes_all[shell_key].append(new_cluster.get_cluster_size())
        num_atoms_all[shell_key].append(num_atoms)
    
    print(f"Processed size: {new_cluster.get_cluster_size()} A with {len(bond_lengths)} shells")
    # print(new_cluster.CN_summary[f'{center_atom}-{CN_atom}'])
    new_cluster.view_xyz(label=False)


Processed size: 2.774093039679816 A with 1 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

No under-coordinated atoms found.
Processed size: 8.322279119039449 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 13.870465198399081 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 19.418651277758716 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 29.489013509888732 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Polynomial fitting

In [13]:
from neighbor_core import fit_polynomial
import plotly.graph_objects as go

fig = go.Figure()
degree = 6

# Loop through each shell, fit the polynomial, and add traces to the main figure
for i in range(1, 5):
    shell = f'shell_{i}'
    _, data_trace, fit_trace = fit_polynomial(
        np.array(sizes_all[shell]), np.array(CNs_all[shell]),
        degree=degree, xlabel="d", ylabel=f"N{i}", shell_label=shell
    )
    fig.add_traces([data_trace, fit_trace])

fig.update_layout(
    title="Combined Polynomial Fits for Different Shells",
    xaxis_title="d", yaxis_title="N", legend_title="Shells"
)
fig.show()

/Users/juanjuanhuang/Desktop/neighbor/neighbor_core.py:74: RankWarning:

Polyfit may be poorly conditioned

/Users/juanjuanhuang/Desktop/neighbor/neighbor_core.py:74: RankWarning:

Polyfit may be poorly conditioned

/Users/juanjuanhuang/Desktop/neighbor/neighbor_core.py:74: RankWarning:

Polyfit may be poorly conditioned

/Users/juanjuanhuang/Desktop/neighbor/neighbor_core.py:74: RankWarning:

Polyfit may be poorly conditioned



## Fitting using Michaelis Menten

In [17]:
from neighbor_core import fit_michaelis_menten
# Initialize the main Plotly figure
fig = go.Figure()

# Loop through each shell, fit, and add traces to the main figure
for i in range(1,5):
    shell = f'shell_{i}'
    coff, data_trace, ext_trace, fit_trace = fit_michaelis_menten(
        np.array(sizes_all[shell]), np.array(CNs_all[shell]), 
        # max_value=12,
        xlabel="d", ylabel=f"N{i}", label_prefix=shell
    )
    fig.add_traces([data_trace, ext_trace, fit_trace])

# Configure layout and show
fig.update_layout(
    title="Combined Michaelis-Menten Fits for Different Shells",
    xaxis_title="d", yaxis_title="N", legend_title="Shells", yaxis_range=[0, 25]
)
fig.show()

# Example 3: Shrinking Pt xyz file

In [19]:
fc = FileChooser()
display(fc)

FileChooser(path='/Users/juanjuanhuang/Desktop/neighbor', filename='', title='', show_hidden=False, select_des…

In [20]:
cif_cluster = ClusterNeighbor()
if fc.value is not None:
    cif_cluster.load_xyz(path=fc.value)
else:
   cif_cluster.load_xyz(path="test_Pt/Pt_foil_2d_xyz.xyz")
   
cif_cluster.view_xyz(label=False, show_symbol=False)
cif_cluster.get_cluster_size()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

166.49181084505508

In [21]:
center_atom = "Pt"
CN_atom = "Pt"
CN_bond_range = 6
tolerance = 0.1

# Initialize dictionaries to store results
CNs_all = {}
bondlengths_all = {}
num_atoms_all = {}
sizes_all = {}

sizes = np.arange(5, 120, 10)/2 # radius

for size_i in sizes[::-1]:
    cif_cluster.shrink_cluster_size(new_radius=size_i, self_apply=True)
    
    num_atoms = len(cif_cluster.atoms)
    cif_cluster.atoms.set_pbc(False)    
    cif_cluster.get_CN_summary_all(tolerance=tolerance, 
                                   bond_range=CN_bond_range, 
                                   printit=False)
    
    # Get bond lengths for each shell
    bond_lengths = list(cif_cluster.CN_summary[f'{center_atom}-{CN_atom}'].keys())
    cif_cluster.write_to_excel(f"size_{cif_cluster.get_cluster_size():.2f}A")
    # cif_cluster.print_CN_summary()
    for shell_index, bond_length in enumerate(bond_lengths):
        shell_key = f'shell_{shell_index + 1}'
        
        # Ensure the key exists in each dictionary
        if shell_key not in CNs_all:
            CNs_all[shell_key] = []
            bondlengths_all[shell_key] = []
            num_atoms_all[shell_key] = []
            sizes_all[shell_key] = []
        
        # Append data to the relevant shell
        bondlengths_all[shell_key].append(bond_length)
        CNs_all[shell_key].append(cif_cluster.CN_summary[f'{center_atom}-{CN_atom}'][bond_length]['average_CN'])
        sizes_all[shell_key].append(cif_cluster.get_cluster_size())
        num_atoms_all[shell_key].append(num_atoms)
    
    print(f"Processed size: {cif_cluster.get_cluster_size()} A with {len(bond_lengths)} shells")
    # print(cif_cluster.CN_summary[f'{center_atom}-{CN_atom}'])
    cif_cluster.view_xyz(label=False)


Processed size: 114.84879197649056 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 104.75645310401072 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 94.88858132344482 A with 4 shells


...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 14.145154541849308 A with 4 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 3.92316 A with 2 shells


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### polynomial fit

In [ ]:
fig = go.Figure()
degree = 6

# Loop through each shell, fit the polynomial, and add traces to the main figure
for i in range(1, len(sizes_all)+1):
    shell = f'shell_{i}'
    _, data_trace, fit_trace = fit_polynomial(
        np.array(sizes_all[shell]), np.array(CNs_all[shell]),
        degree=degree, xlabel="d", ylabel=f"N{i}", shell_label=shell
    )
    fig.add_traces([data_trace, fit_trace])

fig.update_layout(
    title="Combined Polynomial Fits for Different Shells",
    xaxis_title="d", yaxis_title="N", legend_title="Shells"
)
fig.show()

### Michaelis menten fit

In [41]:
# from neighbor_core import fit_michaelis_menten
# Initialize the main Plotly figure
fig = go.Figure()

# Loop through each shell, fit, and add traces to the main figure
for i in range(1,4):
    shell = f'shell_{i}'
    coff, data_trace, ext_trace, fit_trace = fit_michaelis_menten(
        np.array(sizes_all[shell]), np.array(CNs_all[shell]), 
        # max_value=12,
        xlabel="d", ylabel=f"N{i}", 
        maxfev=50000, extended_range=100,
        label_prefix=shell
    )
    fig.add_traces([data_trace, ext_trace, fit_trace])

# Configure layout and show
fig.update_layout(
    title="Combined Michaelis-Menten Fits for Different Shells",
    xaxis_title="d", yaxis_title="N", legend_title="Shells", yaxis_range=[0, 10]
)
fig.show()

# Example 4: Polynomial fits of 2D oxide expanding

## Select a file

In [42]:
fc = FileChooser()
display(fc)

FileChooser(path='/Users/juanjuanhuang/Desktop/neighbor', filename='', title='', show_hidden=False, select_des…

## View the CIF file

In [43]:
cif_cluster = ClusterNeighbor()
if fc.value is not None:
    cif_cluster.load_xyz(path=fc.value)
else:
   cif_cluster.load_xyz(path="structure_examples/PtO2.cif")
   
cif_cluster.view_xyz(label=True, show_symbol=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### expand it

In [44]:
replication_factor = [10, 10, 1]

expanded = ClusterNeighbor()
expanded.load_xyz(from_file=False, atom_object=cif_cluster.expand_cif(replication_factors=replication_factor, self_apply=False))
expanded.view_xyz(label=False, show_symbol=False)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### shift the cell and expand again

In [47]:
shifted_positions = cif_cluster.atoms.get_scaled_positions() +0.3
cif_cluster.atoms.set_scaled_positions(shifted_positions)
cif_cluster.refresh_atoms()
cif_cluster.view_xyz(label=True, show_symbol=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [48]:
replication_factor =[10, 10, 1]

expanded = ClusterNeighbor()
expanded.load_xyz(from_file=False, atom_object=cif_cluster.expand_cif(replication_factors=[10, 10, 1], self_apply=False))
expanded.view_xyz(label=False, show_symbol=False)

# expanded.get_CN_summary_all(tolerance=0.05, bond_range=5, printit=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Calculation loops

In [49]:
center_atom = "Pt"
CN_atom = "O"
CN_bond_range = 4
tolerance = 0.1
undercoordinated_threshold = 6

# Initialize dictionaries to store results
CNs_all = {}
bondlengths_all = {}
num_atoms_all = {}
sizes_all = {}

replication_list = [[i, i, 1] for i in range(2, 11)]  # Generates rows [1,1,1] to [10,10,1]

for replication_factor_i in replication_list:
    expanded_cluster = cif_cluster.expand_cif(replication_factors=replication_factor_i, self_apply=False)
    new_cluster = ClusterNeighbor()
    new_cluster.load_xyz(from_file=False, atom_object=expanded_cluster)
    new_cluster.view_xyz(label=False)

    num_atoms = len(new_cluster.atoms)
    new_cluster.atoms.set_pbc(False)
    new_cluster.get_CN_summary_all(tolerance=tolerance, 
                                   bond_range=CN_bond_range, 
                                   printit=False)
    
    # Get bond lengths for each shell
    bond_lengths = list(new_cluster.CN_summary[f'{center_atom}-{CN_atom}'].keys())
    new_cluster.write_to_excel(f"size_{new_cluster.get_cluster_size():.2f}A")
    # new_cluster.print_CN_summary()
    for shell_index, bond_length in enumerate(bond_lengths):
        shell_key = f'shell_{shell_index + 1}'
        
        # Ensure the key exists in each dictionary
        if shell_key not in CNs_all:
            CNs_all[shell_key] = []
            bondlengths_all[shell_key] = []
            num_atoms_all[shell_key] = []
            sizes_all[shell_key] = []
        
        # Append data to the relevant shell
        bondlengths_all[shell_key].append(bond_length)
        CNs_all[shell_key].append(new_cluster.CN_summary[f'{center_atom}-{CN_atom}'][bond_length]['average_CN'])
        sizes_all[shell_key].append(new_cluster.get_cluster_size())
        num_atoms_all[shell_key].append(num_atoms)

    print(f"Processed size: {new_cluster.get_cluster_size()} A with {len(bond_lengths)} shells")
    print(new_cluster.CN_summary[f'{center_atom}-{CN_atom}'])


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 8.97276985585819 A with 3 shells
defaultdict(<class 'dict'>, {2.0363653179981065: {'average_CN': 3.75, 'info': '2 atoms have CN of 2; \n2 atoms have CN of 3; \n4 atoms have CN of 5; ', 'tolerance': 0.1}, 3.354728578154096: {'average_CN': 0.5, 'info': '4 atoms have CN of 0; \n4 atoms have CN of 1; ', 'tolerance': 0.1}, 3.760408686762494: {'average_CN': 2.625, 'info': '1 atoms have CN of 1; \n4 atoms have CN of 2; \n1 atoms have CN of 3; \n1 atoms have CN of 4; \n1 atoms have CN of 5; ', 'tolerance': 0.1}})


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 13.363374277687774 A with 3 shells
defaultdict(<class 'dict'>, {2.0363666207486526: {'average_CN': 4.444444444444445, 'info': '2 atoms have CN of 2; \n4 atoms have CN of 3; \n8 atoms have CN of 5; \n4 atoms have CN of 6; ', 'tolerance': 0.1}, 3.354728578154096: {'average_CN': 0.5, 'info': '9 atoms have CN of 0; \n9 atoms have CN of 1; ', 'tolerance': 0.1}, 3.7554601565829735: {'average_CN': 3.9444444444444446, 'info': '1 atoms have CN of 1; \n4 atoms have CN of 2; \n4 atoms have CN of 3; \n2 atoms have CN of 4; \n4 atoms have CN of 5; \n1 atoms have CN of 6; \n2 atoms have CN of 8; ', 'tolerance': 0.1}})


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

...

Processed size: 40.00523669681189 A with 3 shells
defaultdict(<class 'dict'>, {2.0363689432524916: {'average_CN': 5.390625, 'info': '2 atoms have CN of 2; \n14 atoms have CN of 3; \n28 atoms have CN of 5; \n84 atoms have CN of 6; ', 'tolerance': 0.1}, 3.3547285781540954: {'average_CN': 0.5, 'info': '64 atoms have CN of 0; \n64 atoms have CN of 1; ', 'tolerance': 0.1}, 3.7518150576525646: {'average_CN': 6.0234375, 'info': '1 atoms have CN of 1; \n4 atoms have CN of 2; \n19 atoms have CN of 3; \n7 atoms have CN of 4; \n19 atoms have CN of 5; \n36 atoms have CN of 6; \n12 atoms have CN of 8; \n30 atoms have CN of 9; ', 'tolerance': 0.1}})


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 45.40144245250657 A with 3 shells
defaultdict(<class 'dict'>, {2.0363691144773766: {'average_CN': 5.45679012345679, 'info': '2 atoms have CN of 2; \n16 atoms have CN of 3; \n32 atoms have CN of 5; \n112 atoms have CN of 6; ', 'tolerance': 0.1}, 3.354728578154096: {'average_CN': 0.5, 'info': '81 atoms have CN of 0; \n81 atoms have CN of 1; ', 'tolerance': 0.1}, 3.7516333827051382: {'average_CN': 6.179012345679013, 'info': '1 atoms have CN of 1; \n4 atoms have CN of 2; \n22 atoms have CN of 3; \n8 atoms have CN of 4; \n22 atoms have CN of 5; \n49 atoms have CN of 6; \n14 atoms have CN of 8; \n42 atoms have CN of 9; ', 'tolerance': 0.1}})


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Processed size: 50.80352069981504 A with 3 shells
defaultdict(<class 'dict'>, {2.0363692525021975: {'average_CN': 5.51, 'info': '2 atoms have CN of 2; \n18 atoms have CN of 3; \n36 atoms have CN of 5; \n144 atoms have CN of 6; ', 'tolerance': 0.1}, 3.3547285781540954: {'average_CN': 0.5, 'info': '100 atoms have CN of 0; \n100 atoms have CN of 1; ', 'tolerance': 0.1}, 3.7514922355664404: {'average_CN': 6.305, 'info': '1 atoms have CN of 1; \n4 atoms have CN of 2; \n25 atoms have CN of 3; \n9 atoms have CN of 4; \n25 atoms have CN of 5; \n64 atoms have CN of 6; \n16 atoms have CN of 8; \n56 atoms have CN of 9; ', 'tolerance': 0.1}})


### polynomial fit

In [56]:
fig = go.Figure()
degree = 6

# Loop through each shell, fit the polynomial, and add traces to the main figure
for i in range(1, len(sizes_all)+1):
    shell = f'shell_{i}'
    _, data_trace, fit_trace = fit_polynomial(
        np.array(sizes_all[shell]), np.array(CNs_all[shell]),
        degree=degree, xlabel="d", ylabel=f"N{i}", shell_label=shell
    )
    fig.add_traces([data_trace, fit_trace])

fig.update_layout(
    title="Combined Polynomial Fits for Different Shells",
    xaxis_title="d", yaxis_title="N", legend_title="Shells"
)
fig.show()

### Michael mentens fit

In [57]:
fig = go.Figure()

# Loop through each shell, fit, and add traces to the main figure
for i in range(1, len(sizes_all)+1):
    shell = f'shell_{i}'
    coff, data_trace, ext_trace, fit_trace = fit_michaelis_menten(
        np.array(sizes_all[shell]), np.array(CNs_all[shell]), 
        # max_value=12,
        xlabel="d", ylabel=f"N{i}", 
        maxfev=50000, extended_range=100,
        label_prefix=shell
    )
    fig.add_traces([data_trace, ext_trace, fit_trace])

# Configure layout and show
fig.update_layout(
    title="Combined Michaelis-Menten Fits for Different Shells",
    xaxis_title="d", yaxis_title="N", legend_title="Shells", yaxis_range=[0, 10]
)
fig.show()

# ASE cluster magic numbers

In [58]:
from ase.cluster import Cluster
from ase.cluster import Icosahedron

# Create a simple cluster with 13 atoms
atoms = Icosahedron('Cu', noshells=3)

view = ClusterNeighbor()
view.load_xyz(from_file=False, atom_object=atoms)
view.view_xyz(label=True, show_symbol=True)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Wulff construction examples

In [59]:
from ase.cluster.wulff import wulff_construction
import py3Dmol

# Create a Wulff construction for Cu with specific Miller indices and surface energies
atoms = wulff_construction('Cu', surfaces=[(1, 0, 0), 
                                           (1, 1, 1), 
                                           (1, 1, 0)], 
                           energies=[1.0, 1.0, 1.0], 
                           size=60, 
                           structure='fcc')


view = ClusterNeighbor()
view.load_xyz(from_file=False, atom_object=atoms)
view.view_xyz(label=True, show_symbol=True)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol